In [2]:
from transformers import AutoFeatureExtractor, ASTModel
import torch
import librosa
import joblib, pickle

In [ ]:
def model_load(ast_path, scaler_path, pca_path, ocsvm_path):
    feature_extractor = AutoFeatureExtractor.from_pretrained(ast_path)
    model = ASTModel.from_pretrained(ast_path).to('cuda')

    scaler = joblib.load(scaler_path)
    pca = joblib.load(pca_path)
    ocsvm = joblib.load(ocsvm_path)

    return feature_extractor, model, scaler, pca, ocsvm

feature_extractor, model, scaler, pca, ocsvm = model_load("bookbot/distil-ast-audioset", './robustscaler_AST.pkl', './pca_AST.pkl', './ocsvm_AST.pkl')

In [4]:
data_path = './train/대화_57_43.wav'
audio, sr = librosa.load(data_path, sr=16000)

def inference(audio, sr=16000):
    input_tensor = feature_extractor(audio, sampling_rate=sr, return_tensors="pt")
    with torch.inference_mode():
        feature = model(**input_tensor.to('cuda')).last_hidden_state.detach()
        feature = feature.cpu().numpy().reshape(1, -1)
        feature = scaler.transform(feature)
        feature = pca.transform(feature)
        pred = ocsvm.predict(feature)
    
    return pred

import time
s_time = time.time()
pred = inference(audio)
print(time.time()-s_time)

0.10894584655761719


In [5]:
pred

array([1])